In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('0.merged_df.csv')

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
lon_min, lon_max = 1.40, 2.90
lat_min, lat_max = 40.90, 41.90

In [ ]:
df = df.loc[(df.lon >= lon_min) & (df.lon <= lon_max)]
df = df.loc[(df.lat >= lat_min) & (df.lat <= lat_max)]
df

In [ ]:
n_rows, n_columns = 200, 200

lat_min, lat_max = df.lat.min(), df.lat.max()  # rows
lon_min, lon_max = df.lon.min(), df.lon.max()  # columns

diff_lat = lat_max - lat_min
diff_lon = lon_max - lon_min

step_lat = diff_lat / n_rows
step_lon = diff_lon / n_columns

In [ ]:
categories = df['SELECT'].drop_duplicates().to_list()

In [ ]:
starting_point_lat, starting_point_lon = lat_min, lon_min # start from the highest column and the first column

result_log = {}

#starting bbox
bbox_lat_min, bbox_lat_max = starting_point_lat, starting_point_lat + step_lat
bbox_lon_min, bbox_lon_max = starting_point_lon, starting_point_lon + step_lon

for row in range(n_rows):
    # reset starting point of the column axis
    bbox_lon_max = starting_point_lon
    bbox_lon_min = starting_point_lon + step_lon

    row_df = df.loc[(df.lat > bbox_lat_min) & (df.lat <= bbox_lat_max), :]

    for column in range(n_columns):
        cell_df = row_df.loc[(row_df.lon > bbox_lon_min) & (row_df.lon <= bbox_lon_max), :]

        cell_name = 'R' + str(row + 1) + 'C' + str(column + 1)
        cell_values = [row + 1, column +1, bbox_lat_min, bbox_lat_max, bbox_lon_min, bbox_lon_max]

        for category in categories:
            try:
                count = cell_df.SELECT.value_counts()[category]
                cell_values.append(count)

            except:
                cell_values.append(0)

        result_log[cell_name] = cell_values

        #print('LAT:', bbox_lat_min, '   ', bbox_lat_max, '  LON:', bbox_lon_min, '   ', bbox_lon_max, '    COUNT:', cell_df.shape[0])

        bbox_lon_min = bbox_lon_max
        bbox_lon_max += step_lon

    bbox_lat_min = bbox_lat_max
    bbox_lat_max += step_lat

In [ ]:
result_df = pd.DataFrame(result_log).T
result_df.columns = ['row', 'column', 'lat_min', 'lat_max', 'lon_min', 'lon_max'] + categories
result_df

In [ ]:
result_df.to_csv('1.complete_df.csv')

In [ ]:
filtered_df = result_df[result_df[categories].T.sum(0) != 0] # remove the cells that have 0 on every fclass
filtered_df

In [ ]:
filtered_df.to_csv('1.filtered_df.csv')